In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
from src.commits import *

In [2]:
from tqdm.notebook import tqdm
from time import sleep

In [3]:
gr = GitRepository("~/Github/openstack-nova")

In [4]:
date_start = datetime(2015, 2, 4)
date_end = datetime(2019, 6, 15)

In [5]:
all_commit_info = {}

In [6]:
num_commits = 0

for commit in RepositoryMining("~/Github/openstack-nova", since = date_start, to = date_end, only_in_branch = 'master').traverse_commits():
    num_commits += 1
    
print(num_commits)

21707


In [7]:
with tqdm(total = num_commits) as pbar:
    for commit in RepositoryMining("~/Github/openstack-nova", since = date_start, to = date_end, only_in_branch = 'master').traverse_commits():
        pbar.set_postfix({'hash' : commit.hash})
        all_commit_info[commit.hash] = CommitInfo(commit)
        pbar.update(1)
        sleep(0.001)

  0%|          | 0/21707 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
# for commit in RepositoryMining("~/Github/openstack-nova", single='bf3706fe3ad149d6241a54ea1649934ff26269f7').traverse_commits():
#     print(len(commit.modifications))
#     prior_commits = set()
#     for file in commit.modifications:
#         bug_inducing_commits = gr.get_commits_last_modified_lines(commit, file)
#         for commit_hashes in bug_inducing_commits.values():
#                 prior_commits |= commit_hashes
#         print(bug_inducing_commits)
#     print('---unique prior commits---')
#     print(prior_commits)
#     print(len(prior_commits))

2
{'nova/tests/unit/virt/libvirt/test_utils.py': {'d2f89951039e99c7ab4aa11bea5464004265c76b'}}
{'nova/virt/libvirt/utils.py': {'d2f89951039e99c7ab4aa11bea5464004265c76b', '527c452a6fc05a316eb8121796f761af97522772'}}
---unique prior commits---
{'d2f89951039e99c7ab4aa11bea5464004265c76b', '527c452a6fc05a316eb8121796f761af97522772'}
2


Get prior and future info

In [65]:
with tqdm(total=len(num_commits)) as pbar:
    for commit_hash, commit_info in all_commit_info.items():
        pbar.set_postfix({'hash' : commit_hash})
        # Current commit
        curr_commit = gr.get_commit(commit_hash)
        # Prior commits
        prior_commit_hashes = set()
        for file in curr_commit.modifications:
            bug_inducing_commits = gr.get_commits_last_modified_lines(curr_commit, file)
            for commit_hashes in bug_inducing_commits.values():
                prior_commits_hashes |= commit_hashes
        # Values
        num_prior_commits = 0
        prior_authors = set()
        num_prior_commits_bug_fixing = 0
        for prior_commit_hash in prior_commit_hashes:
            # Not in study scope
            if prior_commit_hash not in all_commit_info:
                continue
            # Get prior commit info
            prior_commit_info = all_commit_info[prior_commit_hash]
            num_prior_commits += 1
            prior_authors.add(prior_commit.author)
            num_prior_bug_fixing += 1 if prior_commit.bug_fixing else 0
            # IF current commit is bug fixing, make prior commit bug inducing
            prior_commit_info.num_future_bug_fixing += 1 if curr_commit.bug_fixing else 0
            prior_commit_info.fix_indusing = True if curr_commit.bug_fixing
        # Update attributes
        commit_info.num_prior_commits = num_prior_commits
        commit_info.num_prior_authors = len(prior_authors)
        commit_info.num_prior_commits_bug_fixing = num_prior_commits_bug_fixing
        # Progress bar
        pbar.update(1)
        sleep(0.001)

KeyError: '76dfb4ba9fa0fed1350021591956c4e8143b1ce9'